Notebook author: Elena Gronskaya

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install imagecodecs
!pip install cpbd
!pip install scipy==1.1.0

In [ ]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import os
import cv2
import imageio
import cpbd
from scipy import ndimage

In [ ]:
# get sharpness measure as average image gradient magnitude
# see https://stackoverflow.com/questions/6646371/detect-which-image-is-sharper
def get_sharpness(img, method='gnorm'):

  img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  array = np.asarray(img, dtype=np.int32)

  if method=='gnorm':
    gy, gx = np.gradient(array)
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness = np.average(gnorm)
  elif method=='dnorm':
    dx = np.diff(array)[1:,:] # remove the first row
    dy = np.diff(array, axis=0)[:,1:] # remove the first column
    dnorm = np.sqrt(dx**2 + dy**2)
    sharpness = np.average(dnorm)

  return sharpness

In [ ]:
# Get  directories: 
# - path to save calculated metrics
# list of paths to directories with images for which image sharpness metrics 
# are to be calculated: high-resolution, low-resolution, super-resolution 
# (model predictions), paired with dataset names: [[name, path],[name, path],..]
# Note: make sure to call landsat (low-resolution) dataset "baseline"
# baseline (landsat) images need to be upscaled to hig-res image size

metrics_dir = "PATH WHERE METRICS DATA IS TO BE SAVED"

# list of paths to directories with images for which image sharpness metrics 
# are to be calculated: high-resolution, low-resolution, super-resolution 
# (model predictions)
directory = [["DATASET/MODEL NAME","PATH TO LOW RESOLUTION DIRECTORY 1"],
             ["DATASET/MODEL NAME","PATH TO LOW RESOLUTION DIRECTORY 2"],
             ["ETC.."]]

# Loop through, calculating the sharpness values and the cumulative probability 
# of blur detection (cpbd), putting in a DF
# see https://pypi.org/project/cpbd/#description

sh_df = {}
cpbd_df = {}

for [model_name, directory] in directory:

  files = sorted([f for f in os.listdir(directory) if ('.png' in f or '.tif' in f)])

  sh_list = []
  cpbd_list = []

  for filename in files:

    img = imageio.imread(directory+filename)

    if model_name=='baseline':
      dims = (265*3, 265*3)
      img = cv2.resize(img.astype('float32'), dims, interpolation= cv2.INTER_CUBIC)
   
    sh_idx = get_sharpness(img)
    sh_list.append(sh_idx)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img[img>255] = 255
    cpbd_idx = cpbd.compute(img)
    cpbd_list.append(cpbd_idx)

  print('sharpness and cpbd calculated for '+ model_name)

  sh_df[model_name] = sh_list
  cpbd_df[model_name] = cpbd_list

sh_df = pd.DataFrame(sh_df)
sh_df.to_csv(os.path.join(metrics_dir, 'sh_df.csv')) 
cpbd_df = pd.DataFrame(cpbd_df)
cpbd_df.to_csv(os.path.join(metrics_dir, 'cpbd_df.csv')) 

In [ ]:
import plotly.express as px

sh_df = pd.read_csv(os.path.join(metrics_dir, 'sh_df.csv'), index_col = 0) 
mean_sh_df = sh_df.mean()
fig = px.bar(mean_sh_df, color = mean_sh_df.index,
             color_discrete_sequence=['#3366CC', '#DC3912', '#FF9900', '#FF9900', '#109618', '#109618', '#109618', '#990099', '#990099','#DD4477', '#DD4477'],
             width=600, height=600, labels={"value": "sharpness index", "index": "model"}, error_y =sh_df.sem())
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'},showlegend=False)
fig.show()